# Price Level Analysis for Restaurants Around Universities

## Problem Statement
There are a lot of stereotypes surrounding college students and food. Students are busy and poor and therefore not good eaters - always eating fast food, takeout, and instant ramen. 
<br>
<br>If this is true, then do restaurants around colleges reflect this? Are the restaurants around colleges cheaper on average than the area around them? Are there more "junk food" restaurants, like fast food, pizza, or burger places? 
<br>
<br>We used Yelp API to look at restaurants near several colleges in Pittsburgh and restaurants around the city in general to compare price level and categories. Our hypotheis is that restaurants around colleges are cheaper and have a higher percentage of "junk food" restaurants and other categories of restaurants associated with college students (Asian, Mexican food etc). Then we are going to visualize our results with Bokeh, a Python visualization library.
<br>
<br>This project demonstrates how people can use API and data science to answer business questions. Specifically, it shows how businesses can use API like Yelp's API to do market or competitor research. It is easy to imagine a restaurant using techniques similar to the ones we employed to help them identify how to set a price level given their location, or assess competition in their category.
<br>
<br>And of course, we hope to help students figure out where to get cheap food :^)

## Table of Contents
<a href="#Data-Collection">Data Collection</a>
<br><a href="#Parsing-and-Analyzing-Data">Parsing and Analyzing Data</a>
<br><a href="#Visualizing-Conclusions">Visualizing Conclusions</a>
<br><a href="#Conclusions">Conclusions</a>
<br><a href="#Further-Steps">Further Steps</a>
<br><a href="#References">References</a>

## Data Collection
Before we use the API, we need to define how we are going to gather the data.
<br>
<br>First, we need to define our scope. We will focus on Pittsburgh schools, because that information is more relevant to us and the readers. The area is also familiar to us, so it will be easier to verify if our approach and conclusions are correct. 
<br>
<br>Pittsburgh has a lot of colleges according to <a href="https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Pittsburgh">this list</a>. We will narrow the list down according to these criteria:
- Must be in Pittsburgh (not Indiana, Moon Township, or other areas surrounding Pittsburgh)
- Must be nonprofit - this criterion simply for narrowing down list of schools
- Must offer BA/BS/MA/PHD/MBA (not covering schools that just offer Assoc. or RN). This criterion is simply for narrowing down list of schools
- Must have more than 1 restaurant in a 0.5 mile radius around the school
<br>
<br>Here is our final list of schools (ranked in order of size):

| School | Address   | Lat | Long 
|------|------|------|------
|   University of Pittsburgh  | 4200 Fifth Ave, Pittsburgh, PA 15260|   40.444019  | -79.953590
|   Community College of Allegheny County  | 808 Ridge Ave, Pittsburgh, PA 15212|   40.450057  | -80.014850  
|   Carnegie Mellon University  | 5000 Forbes Ave, Pittsburgh, PA 15213 |   40.443298  | -79.943911
|   Duquesne University | 600 Forbes Ave, Pittsburgh, PA 15282|   40.437427  | -79.990641
|   Point Park University  | 201 Wood St, Pittsburgh, PA 15222 |   40.438337  |  -80.001611
|   Chatham University  | 1 Woodland Rd, Pittsburgh, PA 15232 |   40.448578  |  -79.924262
|   Carlow University  | 3333 Fifth Ave, Pittsburgh, PA 15213|   40.439613  | -79.963914

<br>
We define each college by the address and latitude and longitude it has listed on Google Maps and use those coordinates to query the Yelp API.
<br>
<br>We also split the schools into "big" and "small" schools. For big schools, we use a 1 mile radius around the school as the test group and a 3 mile radius as a control group (areas in the test group will be excluded). For small schools, we use a 0.5 mile radius and 1.5 mile radius. This range is based off our own experience when consideirng food close to campus.

## Parsing and Analyzing Data
An outline of the steps we are performing is:
- Query Yelp API for list of restaurants in a narrow radius and wider radius around X school in Pittsburgh. The restaurants in the narrow radius (test group) will be compared to the restaurants in the wider radius (control group).
- Find the average price level in the test and control group and compare if the differences are statistically significant.
- For each restaurant category we are looking at, find the percent of total restaurants this cateogory comprises and compare these percents between control and test group
- Find if there is a relationship between category and price level

In [3]:
### Imports
import io, json, time, os
import requests
from pathlib import Path
import pandas as pd
from pandas import DataFrame
import numpy as np
import scipy
import math
from collections import Counter, defaultdict

The following two functions (read_api_key and all_restaurants) are modified code from HW1.

In [4]:
### Read Yelp API key

def read_api_key(filepath='api_key.txt'): 
    os.chmod(filepath, 384)
    # Referenced this https://bytes.com/topic/python/answers/156341-how-convert-string-like-777-octal-integer-like-0777-a
    return Path(filepath).read_text().strip()

In [5]:
### Query API, deal with pagination, store data in .json

def all_restaurants(api_key, filename, lat, lon, rad = 3000):
   
    ### pagination 
    paginationOffset = 20
    paginationLimit = 20
    waitTime = 0.5 #seconds
    
    ### accessing page 
    header = {'Authorization': "Bearer %s" % api_key}
    parameters = {'latitude': lat, 'longitude': lon, 'radius': rad,  "offset": 0, "limit": paginationLimit, "categories":"restaurants"} #must have location, not query?
    
    ### final list
    finalList = []
    
    ### Loop through pages until dict is empty
    signal = True
    pageCount = 0
    
    
    while signal == True:
      
        ### Grab page
        currentPaginationOffset = pageCount * paginationOffset
        parameters ["offset"] = currentPaginationOffset
        response = requests.get('https://api.yelp.com/v3/businesses/search', headers=header, params = parameters)
        pageCount = pageCount + 1
        
        ### Grab data
        dictionaryOfBusinesses= json.loads(response.text)

        ### What if the query returns nothing?
        if dictionaryOfBusinesses == {}: 
            print("No response, break out of loop")
            print("query")
            print(query)
            print("response")
            print(response)
            print("response.text")
            print(response.text)
            print("url")
            break        
        
        try:
            countOfBusinesses = len(dictionaryOfBusinesses["businesses"])
        except:
            print("Error handling")
            print(dictionaryOfBusinesses)
            print("print query")
            print(query)
        
        listOfBusinesses = dictionaryOfBusinesses["businesses"]

        ### add data to list
        finalList.extend(listOfBusinesses)
        ### sleep and end loop
        time.sleep(waitTime)
        if countOfBusinesses == 0: signal = False
        if (currentPaginationOffset + paginationLimit) == 1000: signal = False
 
    with open((filename),'w',encoding='utf-8') as f:
        json.dump(finalList, f, ensure_ascii=False, indent=4)
        #https://stackoverflow.com/questions/12309269/how-do-i-write-json-data-to-a-file
    
    print("done")

In [22]:
### Query Yelp for all schools

api_key = read_api_key()
universities = ['PITT', 'CCAC', 'PPU', 'CHA', 'CMU', 'DUQ', 'CAR']
parameters = [(40.444019, -79.953590, 1609), (40.450057, -80.014850, 1609),
             (40.438337, -80.001611, 805), (40.448578, -79.924262, 805),
             (40.443298, -79.943911, 1609), (40.437427,-79.990641, 1609),
             (40.439613,-79.963914, 805)]

for i, u in enumerate(universities):
    f1 = 'restaurants-'+u+'.json'
    f2 = 'control-'+f1
    lat, lon, rad = parameters[i]
    all_restaurants(api_key, f1, lat, lon, rad)
    all_restaurants(api_key, f2, lat, lon, rad*3)

done
done
done
done
done
done
done
done
done
done
done
done
done
done


Here, we define some helper functions for parsing json files that we got from previous steps, and create dataframes for both the test group and control group.

In [6]:
### Parses both test and control and creates df

def get_response_df(path):
    df = pd.read_json(path)
    df = df.loc[~df["price"].isna()]
    df = df.loc[~df["categories"].isna()]
    df['price_level'] = df['price'].str.len() # convert previous levels "$, $$, $$$, $$$$" to numbers
    return df

### Exclude everyone from control group that is in test group

def exclude_some_rows(df_control, df_test):
    # exclude rows from df1 that are in df2
    
    new_df = df_control[~df_control["id"].isin(df_test["id"])]
   
    return new_df

### Pricing Analysis

Now, we calculate average price level for the test group and control group. Then, we use confidence intervals to try to find out if the difference in pricing is statistically significant. 

In [7]:
### Show difference between results more intuitively

def compare_result(test_result, control_result, result_name):
    if test_result > control_result:
        print("==> test %s is %f%% larger than that of control" % (result_name, 100*(test_result-control_result)/control_result))
    else:
        print("==> test %s is %f%% smaller than that of control" % (result_name, 100*(control_result-test_result)/control_result))

### Find statistical significance 

def find_significance(control_col, test_col):
    # control
    control_avg = control_col.mean()
    control_stdev = control_col.std()
    control_count = control_col.count()
    
    # test
    test_avg = test_col.mean()
    test_stdev = test_col.std()
    test_count = test_col.count()
    
    # we will use a 95% confidence 
    conf_int = scipy.stats.norm.interval(0.95, loc = test_avg, scale = test_stdev/math.sqrt(test_count))
    
    print(">>>>>> Pricing Analysis")
    print("\tcontrol group average: %f" % control_avg)
    print("\tcontrol group stdev: %f" % control_stdev)
    print("\tcontrol group count: %f" % control_count)
    print("\ttest group average: %f" % test_avg)
    print("\ttest group stdev: %f" % test_stdev)
    print("\ttest group count: %f" % test_count)
    print("\tconfidence interval low: %f high: %f" % (conf_int[0],conf_int[1]))
    print("==> is statistically significant?")
    print("\t", control_avg < conf_int[0] or control_avg > conf_int[1])
    print("==> is significantly lower?")
    print("\t", control_avg > conf_int[1])
    compare_result(test_avg, control_avg, "average pricing")
    
    return test_avg, control_avg

### Category Analysis

To investigate what potentially leads to different pricing of restaurants around and far from universities, we can analyze categories of these restaurant. We mainly focus on "fast food" restaurants, which in our hypothesis will decrease the overall pricing for a certain area. Also, we look at "Asian", "Mexican", "Indian" and "Vegetarian" foods to figure out if people surrounding univerisities have different preference for restaurants.

One difficult part with this analysis comes from the messy category information given by Yelp API. Yelp has a set of complex and highly overlapped class assignment for restaurants. For example, we not only have "Fast Food", but also get "Burgers", "Pizza", "Sandwiches", "Hot Dogs" and so on. Apparently, there is no way to exhaustly gain all the desired information if we solely count on string parsing with some keywords set by our imagination. To address this, we copied all restaurant categories from [Yelp all category list](https://www.yelp.com/developers/documentation/v3/all_category_list) in the excel file, labelled categories that we want to do further analysis on, and assigned new more straightforward class names to them.

Then, we define some helper functions to conveniently parse category data as follows.

In [8]:
### Parse Excel file of Yelp's categories

def find_indice(string):
    flag = 0 # flag == 1 means: having found the first bracket
    for i in range(len(string)):
        if flag == 0 and string[i] == "(":
            flag = 1
            bracket = i
        if flag == 1 and string[i] == ",":
            comma = i
            return bracket, comma

def read_cate_excel(path="categories.xlsx"):
    df = pd.read_excel(path)
    df = df[df["Decision"] == "Yes"][["Title", "Categories", "Decision"]]
    df["Alias"] = ""
    for index, row in df.iterrows():
        string = row["Title"]
        bracket, comma = find_indice(string)
        df.loc[index, ["Title"]] = string[: bracket-1]
        df.loc[index, ["Alias"]] = string[bracket+1: comma]
    
    return df

read_cate_excel()

,Title,Categories,Decision,Alias
11,Asian Fusion,Asian,Yes,asianfusion
35,Pancakes,Fast Food,Yes,pancakes
39,Burgers,Fast Food,Yes,burgers
47,Canteen,Fast Food,Yes,canteen
56,Chicken Wings,Fast Food,Yes,chicken_wings
...,...,...,...,...
304,Vegan,Vegetarian,Yes,vegan
305,Vegetarian,Vegetarian,Yes,vegetarian
307,Vietnamese,Asian,Yes,vietnamese
308,Waffles,Fast Food,Yes,waffles


Now we count categories and caculate percentage for analysis.

In [9]:
### Find how much of the total number of restaurants each category comprises

def categories_count(data_df, cate_dict):
    count = Counter()
    indice = defaultdict(list)
    for index, row in data_df.iterrows():
        # categories" field from scraped data is a list of dict
        class_set = set() # make sure we do not count classes for multiple times for one restaurant
        for element in row["categories"]:
            alias = element["alias"] 
            if alias in cate_dict:
                class_set.add(cate_dict[alias])
        
        for key in class_set:
            count[key] += 1
            indice[key].append(index)      
    
    return count, indice
        
def getpercents(df, totalcount):
    df['percent'] = df['count']/totalcount
    return df
    
def analyze_categories(testdf, controldf):
    # Construct a dictionary {Alias: Category}
    categoriesdf =  read_cate_excel()
    categoriesdict = {row["Alias"]: row["Categories"] for index, row in categoriesdf.iterrows()}
    
    testcount, testindice = categories_count(testdf, categoriesdict)
    controlcount, controlindice = categories_count(controldf, categoriesdict)
    
    testcount_df = DataFrame.from_dict(testcount, orient='index', columns = ['count'])
    controlcount_df = DataFrame.from_dict(controlcount, orient='index', columns = ['count'])
    
    # find each category percent
    testpercent = getpercents(testcount_df, testdf.shape[0])
    controlpercent = getpercents(controlcount_df, controldf.shape[0])
    
    # find top 5 percent in each category
    print("\n>>>>>> Category Analysis")
    print("Top 5 Categories in Control")
    print(controlpercent.nlargest(5, 'percent'))
    print("---------------------------")
    print("Top 5 Categories in Test")
    print(testpercent.nlargest(5, 'percent'))
    
    # Intuitively show the difference
    compare_result(testpercent.at["Fast Food", "percent"], controlpercent.at["Fast Food", "percent"], "Fast Food ratio value")
    
    return testpercent, controlpercent

### Combine everything together and do the analysis!

In [10]:
def analysis(path_test, path_control):
    df_test = get_response_df(path_test)
    df_control_ori = get_response_df(path_control)
        
    df_control = exclude_some_rows(df_control_ori, df_test)
    
    test_avg, control_avg = find_significance(df_control['price_level'], df_test['price_level'])
    
    df_test_percent, df_control_percent = analyze_categories(df_test, df_control)
    
    compare_result(test_avg, control_avg, "average pricing")
    
    return (df_test, df_control), (df_test_percent, df_control_percent), (test_avg, control_avg)


print("#############################################################################\nUniversity of Pittsburgh")
pitt_df, pitt_df_percent, pitt_stats = analysis('restaurants-PITT.json', 'control-restaurants-PITT.json')

print("#############################################################################\nCommunity College of Allegheny County")
ccac_df, ccac_df_percent, ccac_stats = analysis('restaurants-CCAC.json', 'control-restaurants-CCAC.json')

print("#############################################################################\nCarnegie Mellon University")
cmu_df, cmu_df_percent, cmu_stats = analysis('restaurants-CMU.json', 'control-restaurants-CMU.json')

print("#############################################################################\nDuquesne University")
duq_df, duq_df_percent, duq_stats = analysis('restaurants-DUQ.json', 'control-restaurants-DUQ.json')

print("#############################################################################\nPoint Park University")
ppu_df, ppu_df_percent, ppu_stats = analysis('restaurants-PPU.json', 'control-restaurants-PPU.json')

print("#############################################################################\nChatham University")
cha_df, cha_df_percent, cha_stats = analysis('restaurants-CHA.json', 'control-restaurants-CHA.json')

print("#############################################################################\nCarlow University")
car_df, car_df_percent, car_stats = analysis('restaurants-CAR.json', 'control-restaurants-CAR.json')

#############################################################################
University of Pittsburgh
>>>>>> Pricing Analysis
	control group average: 1.688017
	control group stdev: 0.617030
	control group count: 484.000000
	test group average: 1.386503
	test group stdev: 0.536624
	test group count: 163.000000
	confidence interval low: 1.304123 high: 1.468883
==> is statistically significant?
	 True
==> is significantly lower?
	 True
==> test average pricing is 17.861997% smaller than that of control

>>>>>> Category Analysis
Top 5 Categories in Control
            count   percent
Fast Food     180  0.371901
Asian          80  0.165289
Mexican        26  0.053719
Vegetarian      9  0.018595
Indian          5  0.010331
---------------------------
Top 5 Categories in Test
            count   percent
Fast Food      71  0.435583
Asian          28  0.171779
Mexican         7  0.042945
Indian          7  0.042945
Vegetarian      2  0.012270
==> test Fast Food ratio value is 17.123381% larger

## Visualizing Conclusions

We have the results now, but since it's in text form, it's pretty hard to parse. So, we decided to dedicate a part of our project to learning how to visualize our analysis. Data visualization can be a useful tool for exploring the results of a project like this, and we found it helped us to understand our learnings. Additionally, since our problem is practical, visualization will help us communicate our results in a way that's easy for any viewer to understand. 
<br><br>We are using a Python library called Bokeh and Google Maps API for visualization. 

In [11]:
### Import libraries for visualization

from bokeh.plotting import figure, output_file, show, gmap
from bokeh.io import output_notebook 
from bokeh.models import HoverTool, ColumnDataSource, GMapOptions, FactorRange
from bokeh.tile_providers import CARTODBPOSITRON
from ast import literal_eval

output_notebook()

Loading BokehJS ...

In [12]:
# make a DF with one column of all tests, one of all controls, index is college
labels = ["PITT", "CCAC", "CMU", "DUQ", "PPU", "CHA", "CAR"]
test_avgs = [pitt_stats[0], ccac_stats[0], cmu_stats[0], 
                duq_stats[0], ppu_stats[0], cha_stats[0], car_stats[0]]

control_avgs = [pitt_stats[1], ccac_stats[1], cmu_stats[1], 
                duq_stats[1], ppu_stats[1], cha_stats[1], car_stats[1]]

dfdict = {"labels": labels, "test_avgs": test_avgs, "control_avgs": control_avgs}
df = DataFrame(dfdict)

# Get colors of graph
colorlist = ["#4997F8", "#D6492B",
             "#CEE1F7", "#F7D5CE",
             "#4997F8", "#D6492B",
             "#CEE1F7", "#F7D5CE",
             "#CEE1F7", "#F7D5CE",
             "#CEE1F7", "#F7D5CE",
             "#4997F8", "#D6492B"]             

# Data
data = {
    'categories': df['labels'],
    't': df['test_avgs'], # change to first df
    'c': df['control_avgs'], # change to first df
    'colors': colorlist
}

# Create graph

groups = ['Test Average', 'Control Average']

x = [(category, group) for category in df['labels'] for group in groups]

avgs = sum(zip(data['t'], data['c']), ())

source = ColumnDataSource(data=dict(x=x, avgs=avgs, colors=colorlist))

p = figure(x_range = FactorRange(*x), plot_height=250, title = "Average Price Level Test vs Control" )

p.vbar(x='x', top = 'avgs', fill_color= 'colors', source=source, width=0.9)

p.xaxis.major_label_orientation = "vertical"

h = HoverTool(tooltips = [('Category', '@x'), ('Average Price Level', '@avgs')])
p.add_tools(h)

show(p)

The highlighted columns indicate a school whose test average was significantly lower than the control average. Although restaurants around some schools are cheaper, this is not the case for many of the schools we looked at. 
<br>
<br>Next, we want to find if there's a relationship between the schools that are highlighted by mapping their locations out.

In [13]:
### make a map with radiuses around pgh with prices mapped out

# Get map
map_options = GMapOptions(lat= 40.446914, lng= -79.993812, map_type="roadmap", zoom=12)
p = gmap(read_api_key(filepath='gmaps_api.txt'), map_options, title="Map of Pittsburgh")

# prepare data
universities = ['PITT', 'CCAC', 'PPU', 'CHA', 'CMU', 'DUQ', 'CAR']
parameters = [(40.444019, -79.953590, 1609), (40.450057, -80.014850, 1609),
             (40.438337, -80.001611, 805), (40.448578, -79.924262, 805),
             (40.443298, -79.943911, 1609), (40.437427,-79.990641, 1609),
             (40.439613,-79.963914, 805)]
labels = []
lat = []
lon = []
radius = []
radius_big = []

for i in range(0, len(universities)):
    labels.append(universities[i])
    lat.append(parameters[i][0])
    lon.append(parameters[i][1])
    radius.append(parameters[i][2])
    radius_big.append((parameters[i][2])*3)

test_avgs = [pitt_stats[0], ccac_stats[0],  ppu_stats[0], 
             cha_stats[0], cmu_stats[0], 
             duq_stats[0], car_stats[0]]

control_avgs = [pitt_stats[1], ccac_stats[1], ppu_stats[1],
                cha_stats[1], cmu_stats[1], 
                duq_stats[1], car_stats[1]]

colors_small = ['#4997F8',"#CEE1F7", "#CEE1F7",
               "#CEE1F7",'#4997F8',
               "#CEE1F7",'#4997F8']
colors_big = ["#D6492B","#F7D5CE", "#F7D5CE",
              "#F7D5CE", "#D6492B",
              "#F7D5CE", "#D6492B"]

data = {"labels": labels, "lat": lat, "lon": lon,
       "radius": radius, "radius_big": radius_big,
       "test_avgs": test_avgs, "control_avgs": control_avgs,
       "colors_small": colors_small, "colors_big": colors_big}

# create graph

source = ColumnDataSource(data = data)

small = p.circle(x="lon", y="lat", radius='radius_big', fill_color='colors_big', fill_alpha=0.2, source=source)
big = p.circle(x="lon", y="lat", radius='radius', fill_color='colors_small', fill_alpha=0.5, source=source)

h = HoverTool(tooltips = [('School', '@labels'),
                          ('Test Average', '@test_avgs'),
                          ('Control Average', '@control_avgs')])
p.add_tools(h)

show(p)

This is pretty chaotic, but we can roughly see that the schools in east Pittsburgh (near Oakland) are highlighted. 
<br><br> This is mapped out more clearly below.

In [14]:
map_options = GMapOptions(lat= 40.446914, lng= -79.993812, map_type="roadmap", zoom=12)
p = gmap(read_api_key(filepath='gmaps_api.txt'), map_options, title="Map of Pittsburgh")

radius = [500] * len(labels)

data = {"labels": labels, "lat": lat, "lon": lon,
       "radius": radius,
       "test_avgs": test_avgs, "control_avgs": control_avgs,
       "colors": colors_big}

# create graph

source = ColumnDataSource(data = data)

p.circle(x="lon", y="lat", radius='radius', fill_color='colors', fill_alpha=0.8, source=source)
h = HoverTool(tooltips = [('School', '@labels'),
                          ('Test Average', '@test_avgs'),
                          ('Control Average', '@control_avgs')])
p.add_tools(h)

show(p)

Clearly, the three significant schools are near each other. Potential reasons for the price difference in this area could be that there are two big universities (CMU and PITT) close to each other, so many cheap restaurants are in this area. In contrast, the other schools are more spread out so the area around them may be less "college town"-ish and therefore less cheap.

Another thing we could look at to figure out why these specific schools have lower price levels in their test group is the categories of restaurants around each school. In the previous analysis, we found out that, compared with control group, university areas (PITT, CMU, CAR) with higher percentages of fast food restaurants have lower price levels, while university areas (CCAC, DUQ, CHA, PPU) with relatively low percentages of fast food restaurants have price levels that are more expensive than the area far from it. This seems that percent of fast food near a school is correlated with price level of restaurants.

Now, let's intuitively visualize it.

In [19]:
## compare fast food in test and control group
# take % fast food in test and control

# prep df
labels = ["PITT", "CCAC", "CMU", "DUQ", "PPU", "CHA", "CAR"]
percent_df = [pitt_df_percent, ccac_df_percent, cmu_df_percent,
             duq_df_percent, ppu_df_percent, cha_df_percent,
             car_df_percent]
n = len(labels)
test_p = []
control_p = []

for i in range(0, n):
    testdf = percent_df[i][0]
    controldf = percent_df[i][1]
    test_p.append(testdf.loc['Fast Food', 'percent'])
    control_p.append(controldf.loc['Fast Food', 'percent'])
# Get colors of graph
colorlist = ["#4997F8", "#D6492B",
             "#CEE1F7", "#F7D5CE",
             "#4997F8", "#D6492B",
             "#CEE1F7", "#F7D5CE",
             "#CEE1F7", "#F7D5CE",
             "#CEE1F7", "#F7D5CE",
             "#4997F8", "#D6492B"]  

data = {"labels": labels, "test_p": test_p, "control_p": control_p, "colors": colorlist}

groups = ['Test Percent', 'Control Percent']

x = [(category, group) for category in labels for group in groups]

percents = sum(zip(test_p, control_p), ())

source = ColumnDataSource(data=dict(x=x, percents = percents, colors=colorlist))

p = figure(x_range = FactorRange(*x), plot_height=250, title = "Fast Food Percent Test vs Control" )

p.vbar(x='x', top = 'percents', fill_color= 'colors', source=source, width=0.9)

p.xaxis.major_label_orientation = "vertical"

h = HoverTool(tooltips = [('Category', '@x'), ('Average Perent Fast Food', '@percents')])
p.add_tools(h)

show(p)

Furthermore, we can graph the top 3 categories near each school to gain some insights into food preference of students (or people around colleges). 

In [18]:
### make a bar chart with all colleges + top 3 categories 

# use school_df_percent and find the top 3
# have list- first, second, third and do that for all schools

labels = ["PITT", "CCAC", "CMU", "DUQ", "PPU", "CHA", "CAR"]
percent_df = [pitt_df_percent, ccac_df_percent, cmu_df_percent,
             duq_df_percent, ppu_df_percent, cha_df_percent,
             car_df_percent]
first_p = []
second_p = []
third_p = []

first_label = []
second_label = []
third_label = []

x = []


for i in range(0, len(labels)):
    label = labels[i]
    df = percent_df[i]
    df = df[0] # test
    top3df = df.nlargest(3, 'percent')
    
    first_p.append(top3df.iloc[0]['percent'])
    first_label.append(top3df.iloc[0].name)
    x.append((label, top3df.iloc[0].name))
    
    second_p.append(top3df.iloc[1]['percent'])
    second_label.append(top3df.iloc[1].name)
    x.append((label, top3df.iloc[1].name))
    try:
        third_p.append(top3df.iloc[2]['percent'])
        third_label.append(top3df.iloc[2].name)
        x.append((label, top3df.iloc[2].name))
    except:
        third_p.append(0)
        third_label.append("N/A")
        x.append((label, "N/A"))

        
colors = ["#4997F8"] * (len(labels)*3)


data = {'labels': labels, 
        'first_p': first_p, 'second_p': second_p, 'third_p': third_p,
        'first_label': first_label, 'second_label': second_label, 'third_label': third_label,
       'colors': colors}


percents = sum(zip(data['first_p'], data['second_p'], data['third_p']), ())


source = ColumnDataSource(data=dict(x=x, percents=percents, colors = colors))

p = figure(x_range = FactorRange(*x), plot_height=250, title = "Top 3 Categories Per School by Percent" )

p.vbar(x='x', top = 'percents', fill_color= 'colors', source=source, width=0.9)

p.xaxis.major_label_orientation = "vertical"

h = HoverTool(tooltips = [('Category', '@x'),
                          ('Value', '@percents')])
p.add_tools(h)

show(p)

For most schools, the majority of the restaurants in the area around them are fast food and Asian restaurants, which in some sense demonstrates food preference of students (or people surrounding universities). 
<br>
<br>Here, We use another map to explore the relationship between these categories and prices around one school (specifically CMU) and also make it convenient for readers to explore restaurants around universities with respect to their own food preference!

In [20]:
universities = ['PITT', 'CCAC', 'PPU', 'CHA', 'CMU', 'DUQ', 'CAR']
parameters = [(40.444019, -79.953590, 1609), (40.450057, -80.014850, 1609),
             (40.438337, -80.001611, 805), (40.448578, -79.924262, 805),
             (40.443298, -79.943911, 1609), (40.437427,-79.990641, 1609),
             (40.439613,-79.963914, 805)]
list_of_dfs = [pitt_df, ccac_df, ppu_df, cha_df, cmu_df, duq_df, car_df]

university_dict = {}
for i in range(0, len(universities)):
    university_dict[universities[i]] = (parameters[i], list_of_dfs[i])
    

def visualize_map(university, category, university_dict = university_dict):
    ### category must be string
    ### university must be string
    ### Category must be in categories index
    ### University must be 3-character code
    
    ### First, process the df
    uni_data, uni_dfs= university_dict[university] 

    df = pd.concat([uni_dfs[0], uni_dfs[1]])

    df = df[['id', 'name', 'categories', 'coordinates', 'price_level']]

    # coordinates
    # https://stackoverflow.com/questions/38231591/splitting-dictionary-list-inside-a-pandas-column-into-separate-columns
    df['lat'] = df['coordinates'].apply(pd.Series)['latitude']
    df['lon'] = df['coordinates'].apply(pd.Series)['longitude']


    categoriesdf =  read_cate_excel()
    categoriesdict = {row["Alias"]: row["Categories"] for index, row in categoriesdf.iterrows()}

    count, index = categories_count(df, categoriesdict)
    df['category_wide'] = "Other"
    
    ### Takes category string input
    df.loc[index[category], ['category_wide']] = category

    colorsdict = {"Fast Food":"#cc0000", "Asian":"#ffcc66", "Indian":"#cc66ff", "Mexican":"#33ccff", "Vegetarian":"#33cc33", "Other":"#808080"}

    colorsCol = []
    for index, row in df.iterrows(): 
        cat = row['category_wide']
        colorsCol.append(colorsdict[cat])

    df['colors'] = colorsCol  

    ### Add cols that correspond to university
    df['university_lat'] = uni_data[0]
    df['university_lon'] = uni_data[1]
    df['university_rad'] = uni_data[2]

    ### Now prepare data for the graph

    map_options = GMapOptions(lat= 40.443298, lng= -79.943911, map_type="roadmap", zoom=12)
    p = gmap(read_api_key(filepath='gmaps_api.txt'), map_options, title="Map of Restsaurants Around CMU")
    dot_radius = [100] * len(df)

    data = {"id": df['id'], "name": df['name'], 'category': df['category_wide'], 'lat': df['lat'], 'lon': df['lon'],
           "price_level": df['price_level'], "dot_radius":dot_radius, 'dot_colors': df['colors'], 
            "uni_lat": df['university_lat'], "uni_lon":df['university_lon'], "uni_rad": df['university_rad']}
    source = ColumnDataSource(data = data)
   
    
    # Add circle around university
    p.circle(x="uni_lon", y = "uni_lat", radius = 'uni_rad', fill_color = None, line_color = "blue", source=source)

    # Show dots for each restaurant

    p.circle(x="lon", y="lat", radius='dot_radius', fill_color='dot_colors', line_color = None, fill_alpha=0.8, source=source)
    h = HoverTool(tooltips = [('Restaurant', '@name'),
                              ('Category', '@category'),
                              ('Price Level', '@price_level')])
    p.add_tools(h)

    show(p)

In [21]:
visualize_map("CMU", "Fast Food")
# Change the first input to any school in this list: PITT, CCAC, PPU, CHA, CMU, DUQ, CAR
# Change the second input to any category in this list: Fast Food, Asian, Indian, Mexican, Vegetarian

## Conclusions
We found that surprisingly, our hypothesis was not totally correct. It is not necessarily true that restaurants around universities in Pittsburgh are cheaper than restaurants in Pittsburgh in general. However, we did find that price level of restaurants was correlated with percent of fast food restaurants around a university (higher percentages of fast food restaurants lead to lower price levels). We also found that universities with lower price level restaurants around them were concentrated in Oakland. We think that this is the case because there are two major universities (CMU, UPitt) close together in that area, where in other areas, the unversities are smaller or more spread apart.
<br><br>We are satisfied with this conclusion, but there are more steps that can be taken to expand this analysis.

## Further Steps

If we were to continue with our project, we could expand this question in the following ways:
- Ask if this price and category difference holds for a larger sample size and extend the question to other cities besides Pittsburgh
- Ask in general where the cheapest restaurants are in Pittsburgh
- Ask if the size of the school/income level of students/tuition affects the price level of restaurants around a school
- Ask if the demographics of a school affect the categories of restaurants (for example, does a school with a high percentage of Asian students have lots of Asian restaurants around it?)

## References
<a href="https://pandas.pydata.org/pandas-docs/stable/">Pandas Documentation</a>
<br><a href="https://docs.bokeh.org/en/latest/index.html">Bokeh Documentation</a>
<br><a href="https://blog.minitab.com/blog/adventures-in-statistics-2/understanding-hypothesis-tests-confidence-intervals-and-confidence-levels">Confidence Interval Article</a>
<br><a href="https://stackoverflow.com/questions/28242593/correct-way-to-obtain-confidence-interval-with-scipy">Confidence Intervals with Scipy</a>